In [21]:
import spacy
from spacy.util import minibatch, compounding
from spacy.training.example import Example
from json import load
from string import punctuation
from random import shuffle
nlp = spacy.load("ru_core_news_sm")


In [22]:
type(nlp)

spacy.lang.ru.Russian

In [23]:
def to_describer(text, ent):
    return (text, {"entities": ent})


In [24]:
with open('../../Data/types_of_events.json', 'r', encoding='utf-8') as f:
    dataset_of_types = load(f)
ner = nlp.get_pipe("ner")
punctuation = list(map(lambda a: a, punctuation)) + \
    ['(', ')', '"', '«', '»', "'"]


In [25]:
print(len('Мастер-класс'))


12


In [26]:
TRAIN_DATA = []
for i in dataset_of_types:
  for j in dataset_of_types[i]:
    x = i.find(j)
    if x == -1:
      x+=1
    TRAIN_DATA.append(to_describer(i, [(x, x+len(j), "type")]))
len(TRAIN_DATA)


134

In [27]:
list(filter(lambda a: a[1]['entities'][0][0] == -1, TRAIN_DATA))


[]

In [28]:
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [
    pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

shuffle(TRAIN_DATA)

losses = {}
for batch in spacy.util.minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001)):
    for text, annotations in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], losses=losses, drop=0.3)


c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Трекшн-митинг для команд Акселератора «Стартап-пол..." with entities "[(0, 12, 'type')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Мастер-класс по подготовке к питчу и созданию итог..." with entities "[(0, 13, 'type')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [35]:
doc1 = nlp('Квест «Омские легенды»')
doc2 = nlp('Информационная конференция по ИИ')
print(doc1.ents, doc2.ents)


(Квест,) (конференция,)


In [37]:
nlp.to_disk('../../PipeLines/WordsExtraction/words_extract')